In [16]:
import pandas as pd
import numpy as np
import pickle

In [17]:
sp = pd.read_csv("E:/baselines/ZZ500.csv")
# sp = sp.drop(columns=['prev_close' ,'adjfactor'])
sp['dt'] = pd.to_datetime(sp['dt'])

In [18]:
grouped_df = sp.groupby('kdcode')
l = sp['dt'].nunique()

def filter_groups(group):
    return len(group) == l

df = grouped_df.filter(filter_groups)

In [19]:
df['kdcode'].nunique()

116

In [20]:
df['label'] = df.groupby('kdcode')['close'].pct_change()

In [21]:
df['label'] = df['label'].fillna(0)
df

,kdcode,dt,close,open,high,low,turnover,volume,label
0,000009.SZ,2018-01-02,7.32,7.27,7.33,7.23,140497530.0,19269993.0,0.000000
1,000009.SZ,2018-01-03,7.42,7.32,7.44,7.30,170636551.0,23092097.0,0.013661
2,000009.SZ,2018-01-04,7.64,7.52,7.70,7.51,358089946.0,46992707.0,0.029650
3,000009.SZ,2018-01-05,7.55,7.65,7.65,7.51,196259451.0,25990264.0,-0.011780
4,000009.SZ,2018-01-08,7.52,7.55,7.63,7.46,178511079.0,23703431.0,-0.003974
...,...,...,...,...,...,...,...,...,...
169007,603883.SH,2023-12-25,29.37,29.02,29.58,28.80,101007153.0,3457490.0,0.012061
169008,603883.SH,2023-12-26,29.05,29.26,29.49,28.82,66158144.0,2272587.0,-0.010895
169009,603883.SH,2023-12-27,30.08,28.94,30.40,28.80,206474442.0,6939076.0,0.035456
169010,603883.SH,2023-12-28,28.98,30.15,30.20,28.44,264520627.0,9053474.0,-0.036569


In [22]:
# 先创建一个空的 MultiIndex
index = pd.MultiIndex(levels=[[], []], codes=[[], []], names=['dt', 'kdcode'])

# 遍历每个公司，为每个日期添加公司，并更新 MultiIndex
for company in df['kdcode'].unique():
    # 为每个公司创建一个临时 MultiIndex
    temp_index = pd.MultiIndex.from_product([[date for date in df['dt'].unique()], [company]], names=['dt', 'kdcode'])
    # 更新 MultiIndex
    index = index.union(temp_index)
myData = pd.DataFrame(df[['close', 'high', 'low', 'open', 'volume', 'turnover']].values, index=index, columns=['close', 'high', 'low', 'open', 'volume', 'turnover'])

In [23]:
myData

close   high    low   open      volume     turnover
dt         kdcode                                                        
2018-01-02 000009.SZ   7.32   7.33   7.23   7.27  19269993.0  140497530.0
           000021.SZ   7.42   7.44   7.30   7.32  23092097.0  170636551.0
           000027.SZ   7.64   7.70   7.51   7.52  46992707.0  358089946.0
           000031.SZ   7.55   7.65   7.51   7.65  25990264.0  196259451.0
           000039.SZ   7.52   7.63   7.46   7.55  23703431.0  178511079.0
...                     ...    ...    ...    ...         ...          ...
2023-12-29 603355.SH  29.37  29.58  28.80  29.02   3457490.0  101007153.0
           603444.SH  29.05  29.49  28.82  29.26   2272587.0   66158144.0
           603568.SH  30.08  30.40  28.80  28.94   6939076.0  206474442.0
           603868.SH  28.98  30.20  28.44  30.15   9053474.0  264520627.0
           603883.SH  29.89  30.00  28.80  28.97   6878338.0  204020452.0

[169012 rows x 6 columns]

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(myData)
scaled_myData = pd.DataFrame(scaled_data, index=myData.index, columns=myData.columns)

In [25]:
scaled_myData

close      high       low      open    volume  \
dt         kdcode                                                        
2018-01-02 000009.SZ  0.009366  0.010914  0.011568  0.011100  0.014217   
           000021.SZ  0.009519  0.011078  0.011680  0.011176  0.017037   
           000027.SZ  0.009856  0.011465  0.012016  0.011482  0.034671   
           000031.SZ  0.009718  0.011391  0.012016  0.011680  0.019175   
           000039.SZ  0.009672  0.011361  0.011936  0.011528  0.017488   
...                        ...       ...       ...       ...       ...   
2023-12-29 603355.SH  0.043164  0.044044  0.046079  0.044309  0.002551   
           603444.SH  0.042674  0.043910  0.046111  0.044675  0.001677   
           603568.SH  0.044253  0.045265  0.046079  0.044187  0.005120   
           603868.SH  0.042567  0.044967  0.045503  0.046034  0.006680   
           603883.SH  0.043961  0.044669  0.046079  0.044232  0.005075   

                      turnover  
dt         kdcode               
2018-01-02 000009.SZ  0.012905  
           000021.SZ  0.015673  
           000027.SZ  0.032890  
           000031.SZ  0.018026  
           000039.SZ  0.016396  
...                        ...  
2023-12-29 603355.SH  0.009277  
           603444.SH  0.006077  
           603568.SH  0.018964  
           603868.SH  0.024296  
           603883.SH  0.018739  

[169012 rows x 6 columns]

In [26]:
scaled_myData.to_pickle('zzfea.pkl')

In [27]:
labData = pd.DataFrame(df[['label']].values, index=index, columns=['label'])
labData_filled = labData.fillna(0)

In [28]:
labData_filled

label
dt         kdcode             
2018-01-02 000009.SZ  0.000000
           000021.SZ  0.013661
           000027.SZ  0.029650
           000031.SZ -0.011780
           000039.SZ -0.003974
...                        ...
2023-12-29 603355.SH  0.012061
           603444.SH -0.010895
           603568.SH  0.035456
           603868.SH -0.036569
           603883.SH  0.031401

[169012 rows x 1 columns]

In [29]:
labData_filled.to_pickle('zzlab.pkl')